In [1]:
import requests
import json
import time
import re
from typing import List, Dict, Optional
from html import unescape
from bs4 import BeautifulSoup

In [2]:
import praw
import pandas as pd
# from praw.models import MoreComments #This import is needed is used only if we want to limit the comments we scrape

# #URL to access the app needed to scrape the data off the Berkeley subreddit
# # https://www.reddit.com/prefs/apps

#This is the tutorial I used to set up the web scraping using PRAW
#https://www.geeksforgeeks.org/python/scraping-reddit-using-python/

# #PLAN
# #Scrape the data off the Berkeley subreddit about consulting clubs
# #Organize that data using the pandas library
# #Create chatbot that utilizes natural language processing that will give users feedback
# #about each consulting club here at Berkeley. 

reddit_read_only = praw.Reddit(client_id = "QlBfNfxQ3e_MGP9RkaOQig",
                               client_secret = "SpLjOwYdQPU4z1wqcXBjVl_7DnUIZg",
                               user_agent = "Berkeley_Consulting")

subreddit = reddit_read_only.subreddit("berkeley")

In [3]:
consulting_posts = subreddit.search('clubs', limit=None)

posts_dict = {'Title': [], 'Post Text': [], 'ID': [], 'Score': [], 'Total Comments': [], 'Post URL': []}
comments_dict = {'Comment': [], 'Score': []}

In [4]:
for post in consulting_posts:
    if len(post.selftext) > 20:
        #The title of the post
        posts_dict['Title'].append(post.title)
        #The text inside of the post
        posts_dict['Post Text'].append(post.selftext)
        #Unique ID of each post
        posts_dict['ID'].append(post.id)
        #The scoure of a post
        posts_dict['Score'].append(post.score)
        #Total number of comments inside the post
        posts_dict['Total Comments'].append(post.num_comments)
        #URL of each post
        posts_dict['Post URL'].append(post.url)
    else:
        continue

consulting_club_posts = pd.DataFrame(posts_dict)
# print(consulting_club_posts)
consulting_club_posts

,Title,Post Text,ID,Score,Total Comments,Post URL
0,Looking for people to start a bridge troll clu...,"Hello, \n\nI was wondering if anyone else woul...",1nfcxn2,265,36,https://www.reddit.com/r/berkeley/comments/1nf...
1,Heartbreaking club experience as a freshman,I applied to 9 tech clubs (i’m an eecs major) ...,1nh1egm,131,52,https://www.reddit.com/r/berkeley/comments/1nh...
2,Berkeley club decisions today got me like:,[Berkeley club decisions today got me like:](h...,1na5tea,71,50,https://www.reddit.com/r/berkeley/comments/1na...
3,Potential nepotism in prestigious clubs is sic...,Title. So exhausted of this kinda stuff. Heard...,1nbpwed,108,36,https://www.reddit.com/r/berkeley/comments/1nb...
4,Asian monoculture in consulting clubs,Over the past month on this subreddit there ha...,16o5u8z,268,112,https://www.reddit.com/r/berkeley/comments/16o...
...,...,...,...,...,...,...
204,So you got rejected from a club. What now?,So you got rejected by a club at Berkeley. Pro...,sh9utu,161,24,https://www.reddit.com/r/berkeley/comments/sh9...
205,Are there any clubs on campus for students who...,Basically what the title says! Or if there are...,1h0w654,3,0,https://www.reddit.com/r/berkeley/comments/1h0...
206,What are some fun clubs on campus?,Anyone have any recommendations?,198ng9e,14,16,https://www.reddit.com/r/berkeley/comments/198...
207,How to get into Consulting clubs?,Hello! I'm going to be a sophomore next year a...,1ek2g0a,4,6,https://www.reddit.com/r/berkeley/comments/1ek...


In [5]:
#Need to figure out a way to get the URL of each and every post
for i in posts_dict['Post URL']:
    if 'comments' in i:
        submission = reddit_read_only.submission(url = i)
    else:
        continue
    for comment in submission.comments:
        if len(comment.body) < 20:
            continue
        comments_dict['Comment'].append(comment.body)
        comments_dict['Score'].append(comment.score)

# print(post_comments[0]) #This only prints one comment right now for one URL. Need to get as many comments as possible for one given URL.

comments_df = pd.DataFrame(comments_dict)
comments_df

#Generate the CSV file that contains all the posts about Berkeley clubs
# comments_df.to_csv('my_dataframe.csv', index=False) # index=False prevents writing the DataFrame index as a column

#They are not marked in any way so they are just random pieces of information
#Need to find a way to associate either the post title or the post URL with the post comments

,Comment,Score
0,Don’t forget riddles,67
1,"if you’re deadass, i will absolutely join",51
2,"I'm actually serious about starting it, if som...",49
3,"definitely don’t go barefoot under a bridge, t...",16
4,Do we get to eat billy goats?,12
...,...,...
1043,Hi! Student here looking for a club!\n\n1. I w...,2
1044,**JOIN THE BERKELEY FORUM**\n\n1. **All Majors...,2
1045,"Hi all, we're Berkeley Phi Beta Lambda, a chap...",2
1046,Hi Everyone! I'm representing Womxn in Math at...,2


In [6]:
new_df = pd.concat([consulting_club_posts['Post Text'], comments_df['Comment']], axis=1)
new_df

,Post Text,Comment
0,"Hello, \n\nI was wondering if anyone else woul...",Don’t forget riddles
1,I applied to 9 tech clubs (i’m an eecs major) ...,"if you’re deadass, i will absolutely join"
2,[Berkeley club decisions today got me like:](h...,"I'm actually serious about starting it, if som..."
3,Title. So exhausted of this kinda stuff. Heard...,"definitely don’t go barefoot under a bridge, t..."
4,Over the past month on this subreddit there ha...,Do we get to eat billy goats?
...,...,...
1043,NaN,Hi! Student here looking for a club!\n\n1. I w...
1044,NaN,**JOIN THE BERKELEY FORUM**\n\n1. **All Majors...
1045,NaN,"Hi all, we're Berkeley Phi Beta Lambda, a chap..."
1046,NaN,Hi Everyone! I'm representing Womxn in Math at...


In [77]:
# #Loading the stored dataframes from the previous notebook using the pickle module
# consulting_club_posts.to_pickle('consulting_club_posts.pkl')
# comments_df.to_pickle('comments_df.pkl')

In [7]:
#!/usr/bin/env python3
"""
CalLink Deep Scraper
Fetches the organization list AND visits each club's individual page
to get comprehensive information
"""

def strip_html(text: str) -> str:
    """Remove HTML tags and clean up text"""
    if not text or not isinstance(text, str):
        return ''
    
    text = re.sub(r'<[^>]+>', '', text)
    text = unescape(text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    
    return text

In [8]:
def fetch_organization_list() -> List[Dict]:
    """Fetch the list of all organizations from the API"""
    base_url = "https://callink.berkeley.edu/api/discovery/search/organizations"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36',
        'Accept': 'application/json',
        'Referer': 'https://callink.berkeley.edu/Organizations',
    }
    
    all_organizations = []
    skip = 0
    top = 100
    
    print("📋 Fetching organization list from CalLink API...")
    print("=" * 70)
    
    while True:
        params = {
            'top': top,
            'skip': skip,
            'orderBy[0]': 'UpperName asc',
            'query': '',
        }
        
        try:
            print(f"  Fetching organizations {skip + 1} to {skip + top}...", end=" ")
            response = requests.get(base_url, params=params, headers=headers, timeout=15)
            
            if response.status_code != 200:
                print(f"❌ Error: Status code {response.status_code}")
                break
            
            data = response.json()
            
            if isinstance(data, dict):
                orgs = data.get('value', data.get('items', data.get('results', [])))
            elif isinstance(data, list):
                orgs = data
            else:
                print("❌ Unexpected response format")
                break
            
            if not orgs:
                print("✓ Done")
                break
            
            all_organizations.extend(orgs)
            print(f"✓ Got {len(orgs)} (Total: {len(all_organizations)})")
            
            if len(orgs) < top:
                break
            
            skip += top
            time.sleep(0.3)
            
        except Exception as e:
            print(f"❌ Error: {e}")
            break
    
    print(f"\n✓ Found {len(all_organizations)} organizations total\n")
    return all_organizations

In [9]:
def scrape_organization_page(org_url: str, org_name: str) -> Optional[Dict]:
    """
    Scrape detailed information from an individual organization page
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36',
    }
    
    try:
        response = requests.get(org_url, headers=headers, timeout=10)
        
        if response.status_code != 200:
            return None
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        details = {}
        
        # Try to extract various fields from the page
        # These selectors are educated guesses - we'll need to adjust based on actual HTML
        
        # Description (usually in a main content area)
        desc_selectors = [
            'div.organization-description',
            'div.description',
            'div[class*="about"]',
            'div[class*="description"]',
            'section.description',
        ]
        for selector in desc_selectors:
            elem = soup.select_one(selector)
            if elem:
                details['full_description'] = strip_html(elem.get_text())
                break
        
        # Purpose/Mission
        purpose_selectors = [
            'div.purpose',
            'div[class*="mission"]',
            'div[class*="purpose"]',
        ]
        for selector in purpose_selectors:
            elem = soup.select_one(selector)
            if elem:
                details['purpose'] = strip_html(elem.get_text())
                break
        
        # Meeting info
        meeting_selectors = [
            'div.meeting-info',
            'div[class*="meeting"]',
            'span[class*="meeting"]',
        ]
        for selector in meeting_selectors:
            elem = soup.select_one(selector)
            if elem:
                details['meeting_info'] = strip_html(elem.get_text())
                break
        
        # Contact email(s)
        emails = soup.find_all('a', href=re.compile(r'mailto:'))
        if emails:
            details['emails'] = ', '.join([e.get('href', '').replace('mailto:', '') for e in emails])
        
        # Social media links
        social_links = {}
        social_patterns = {
            'facebook': r'facebook\.com',
            'instagram': r'instagram\.com',
            'twitter': r'twitter\.com|x\.com',
            'linkedin': r'linkedin\.com',
            'discord': r'discord\.(gg|com)',
        }
        
        for platform, pattern in social_patterns.items():
            link = soup.find('a', href=re.compile(pattern))
            if link:
                social_links[platform] = link.get('href', '')
        
        if social_links:
            details['social_media'] = json.dumps(social_links)
        
        # Website
        website = soup.find('a', href=re.compile(r'^https?://(?!callink\.berkeley)'))
        if website:
            details['website'] = website.get('href', '')
        
        # Try to get all text content as fallback
        if not details:
            main_content = soup.find('main') or soup.find('body')
            if main_content:
                details['full_content'] = strip_html(main_content.get_text())
        
        return details if details else None
        
    except Exception as e:
        return None

In [10]:
def scrape_all_organizations_detailed(org_list: List[Dict], max_orgs: Optional[int] = None) -> List[Dict]:
    """
    Visit each organization's page and collect detailed information
    
    Args:
        org_list: List of organizations from the API
        max_orgs: Optional limit for testing (scrape only first N orgs)
    """
    print("🔍 Scraping detailed information from each organization page...")
    print("=" * 70)
    
    if max_orgs:
        print(f"⚠️  TEST MODE: Only scraping first {max_orgs} organizations\n")
        org_list = org_list[:max_orgs]
    
    enriched_orgs = []
    failed_count = 0
    
    for idx, org in enumerate(org_list, 1):
        # Get basic info from API response
        org_data = {
            'name': strip_html(org.get('Name', org.get('name', ''))),
            'short_description': strip_html(org.get('Description', org.get('description', ''))),
            'status': strip_html(org.get('Status', '')),
            'email': org.get('Email', org.get('email', '')),
        }
        
        # Get categories
        if 'CategoryNames' in org:
            cats = org['CategoryNames']
            org_data['categories'] = ', '.join([strip_html(str(c)) for c in cats]) if isinstance(cats, list) else strip_html(str(cats))
        
        # Build profile URL
        website_key = org.get('WebsiteKey', '')
        if website_key:
            org_data['profile_url'] = f"https://callink.berkeley.edu/organization/{website_key}"
        else:
            org_data['profile_url'] = ''
        
        # Scrape detailed page
        print(f"[{idx}/{len(org_list)}] {org_data['name'][:50]}...", end=" ")
        
        if org_data['profile_url']:
            details = scrape_organization_page(org_data['profile_url'], org_data['name'])
            
            if details:
                org_data.update(details)
                print("✓")
            else:
                print("⚠️  (basic info only)")
                failed_count += 1
        else:
            print("❌ (no URL)")
            failed_count += 1
        
        enriched_orgs.append(org_data)
        
        # Be respectful to the server
        time.sleep(0.5)
        
        # Progress update every 50 orgs
        if idx % 50 == 0:
            print(f"\n  Progress: {idx}/{len(org_list)} completed ({failed_count} failed)\n")
    
    print(f"\n✓ Completed! Successfully scraped {len(enriched_orgs) - failed_count}/{len(enriched_orgs)} organizations")
    print(f"  ({failed_count} organizations had limited/no detailed info)\n")
    
    return enriched_orgs

In [11]:
def save_data(orgs: List[Dict], prefix: str = 'callink_complete'):
    """Save scraped data to CSV and JSON"""
    if not orgs:
        print("❌ No data to save")
        return
    
    df = pd.DataFrame(orgs)
    
    # Save CSV
    csv_file = f'{prefix}.csv'
    df.to_csv(csv_file, index=False)
    print(f"✓ Saved CSV: {csv_file}")
    
    # Save JSON
    json_file = f'{prefix}.json'
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(orgs, f, indent=2, ensure_ascii=False)
    print(f"✓ Saved JSON: {json_file}")
    
    # Print summary
    print("\n" + "=" * 70)
    print("📊 DATA SUMMARY")
    print("=" * 70)
    print(f"Total organizations: {len(df)}")
    print(f"\nColumns ({len(df.columns)}):")
    for col in df.columns:
        non_null = df[col].notna().sum()
        pct = (non_null / len(df)) * 100
        print(f"  • {col}: {non_null}/{len(df)} ({pct:.1f}%)")
    
    print(f"\nSample organizations:")
    print(df[['name', 'categories']].head(10).to_string(index=False))

In [12]:
def main():
    print("\n" + "=" * 70)
    print("🐻 BERKELEY CALLINK DEEP SCRAPER")
    print("=" * 70)
    print("This will scrape BOTH the organization list AND each club's page")
    print("=" * 70 + "\n")
    
    # Step 1: Get organization list
    org_list = fetch_organization_list()
    
    if not org_list:
        print("❌ Failed to fetch organization list")
        return
    
    # Ask user if they want to test first
    print(f"Found {len(org_list)} organizations.")
    print("\nOptions:")
    print("  1. Test mode - scrape first 10 organizations (recommended for testing)")
    print("  2. Scrape ALL organizations (will take 10-20 minutes)")
    print()
    
    # For non-interactive mode, default to test
    # In interactive mode, you can uncomment the input() line
    # choice = input("Enter choice (1 or 2): ").strip()
    #***************************************************
    #CHANGE THIS TO 1 
    choice = "0"  # Default to test mode
    #***************************************************
    
    if choice == "1":
        max_orgs = 10
        print(f"\n🧪 TEST MODE: Scraping first {max_orgs} organizations\n")
    else:
        max_orgs = None
        print(f"\n🚀 FULL MODE: Scraping all {len(org_list)} organizations\n")
        print("⏱️  This will take approximately 10-20 minutes...")
        print("    (Visiting ~1000 pages with 0.5s delay between requests)\n")
    
    # Step 2: Scrape detailed info
    enriched_orgs = scrape_all_organizations_detailed(org_list, max_orgs=max_orgs)
    
    # Step 3: Save data
    print("\n💾 Saving data...")
    prefix = 'callink_test' if max_orgs else 'callink_complete'
    save_data(enriched_orgs, prefix=prefix)
    
    print("\n" + "=" * 70)
    print("✅ SCRAPING COMPLETE!")
    print("=" * 70)
    
    if max_orgs:
        print("\n💡 TIP: This was a test run. To scrape all organizations,")
        print("   change 'choice = \"1\"' to 'choice = \"2\"' in the script.")


if __name__ == "__main__":
    # main()
    pass

In [17]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

URL = "https://eecs.berkeley.edu/people/students-2/organizations/"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/122.0.0.0 Safari/537.36"
}

# --- 1. Download page ---
resp = requests.get(URL, headers=headers)
resp.raise_for_status()
html = resp.text

soup = BeautifulSoup(html, "html.parser")

# --- 2. Helper: find sections we care about ---
section_titles = [
    "EECS organizations",
    "EE organizations",
    "CS organizations",
]

rows = []

for section_title in section_titles:
    # find the <h2> (or similar) whose text matches the section title
    header = soup.find(
        lambda tag: tag.name in ["h2", "h3"]
        and tag.get_text(strip=True) == section_title
    )
    if header is None:
        continue

    # Walk over siblings until the next h2 (i.e., next big section)
    sibling = header.find_next_sibling()
    while sibling and sibling.name != "h2":
        # Each club is under an h3 (or sometimes h4)
        if sibling.name in ["h3", "h4"]:
            club_header = sibling

            full_title = club_header.get_text(" ", strip=True)
            # Try to pull acronym in parentheses at end of title
            m = re.search(r"\(([^)]+)\)\s*$", full_title)
            acronym = m.group(1) if m else ""

            # Main club name = title with trailing "(ABC)" removed
            club_name = full_title
            if m:
                club_name = full_title[:m.start()].strip(", ").strip()

            # Website = first link in the header, if any
            link_tag = club_header.find("a")
            website = link_tag["href"] if link_tag and link_tag.has_attr("href") else ""

            # Collect description + emails from following siblings
            desc_parts = []
            emails = set()

            desc_sib = club_header.find_next_sibling()
            while desc_sib and desc_sib.name not in ["h3", "h4", "h2"]:
                text = desc_sib.get_text(" ", strip=True)
                if text:
                    desc_parts.append(text)

                    # find email-like strings in the text
                    for email in re.findall(r"[\w\.-]+@[\w\.-]+\.\w+", text):
                        emails.add(email)

                desc_sib = desc_sib.find_next_sibling()

            description = " ".join(desc_parts)

            rows.append(
                {
                    "category": section_title,
                    "club_name": club_name,
                    "acronym": acronym,
                    "website": website,
                    "emails": ";".join(sorted(emails)),
                    "description": description,
                }
            )

        sibling = sibling.find_next_sibling()

# --- 3. Save to CSV ---
df = pd.DataFrame(rows)
df.to_csv("data/berkeley_eecs_orgs.csv", index=False)

df.head()

,category,club_name,acronym,website,emails,description
0,EECS organizations,The Association of Women in EE & CS,AWE,http://awe.berkeley.edu,aweberkeley@gmail.com;biasbusters-admin@lists....,Formerly AUWICSEE. aweberkeley@gmail.com AWE i...
1,EECS organizations,Blockchain at Berkeley,,https://blockchain.berkeley.edu/,admin@blockchain.berkeley.edu,admin@blockchain.berkeley.edu Blockchain at Be...
2,EECS organizations,Cloud at Cal,,https://sites.google.com/berkeley.edu/cloudatcal,cloudatcal@gmail.com,cloudatcal@gmail.com Cloud at California is UC...
3,EECS organizations,EECS Transfers at Berkeley,,https://callink.berkeley.edu/organization/eecs...,,EECS Transfers at Berkeley aims to build a com...
4,EECS organizations,Eta Kappa Nu (HKN) EECS Honor Society,,http://hkn.eecs.berkeley.edu/,hkn@hkn.eecs.berkeley.edu,hkn@hkn.eecs.berkeley.edu Eta Kappa Nu (HKN) i...


In [84]:
# new_df.to_csv('posts_and_comments.csv', index=False)